## Loading the pretrained Inception V3 model

In [1]:
from keras.applications import inception_v3
from keras import backend as k

# Disables all trainning-especif operations
k.set_learning_phase(0)

# Builds the Inception V3 network, without its convolutional base. The model will be loaded with pretrained ImageNet weights. 
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)


Using TensorFlow backend.


In [2]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

## Setting up the DeepDream configuration

Dictionary mapping layer names to a coefficient quantifying how mush the layer's activation contributes to the loss we will seek to maximize.

_Note:_ The layer names are hardcoded in the built-in Inception V3 application.

__Layers that are lower in the network contain more-local__, less-abstract
representations and lead to dream patterns that look more geometric. 

__Layers that are higher up__ lead to more-recognizable visual patterns based on the most common
objects found in ImageNet, such as dog eyes, bird feathers, and so on.

In [64]:
layer_contributions = {
    'mixed0': 0.,    
    'mixed1': 0.,    
    'mixed2': 0.,
    'mixed3': 0.,
    'mixed4': 0.,
    'mixed5': 0.,    
    'mixed6': 0.,
    'mixed7': 0.,
    'mixed8': 0.,
    'mixed9': 0.,
    'mixed9_1': 0.,
    'mixed10': 3.,    
}

## Defining the loss to be maximized

In [65]:
# Creates a dictionary that maps layer names to layer instances
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# We will define the loss by adding layer contributions to this scalar variable
# Adds the L2 norm of the features of a layer to the loss. 

loss = k.variable(0.)
for layer_name in layer_contributions:
    
    coeff = layer_contributions[layer_name]

    # Retrieves the layer’s output
    activation = layer_dict[layer_name].output

    scaling = k.prod(k.cast(k.shape(activation), 'float32'))
    
    # Note: Doing activation[:, 2: -2, 2: -2, :] we avoid border artifacts 
    # by only involving nonborder pixels in the loss.
    #loss = loss + coeff * k.sum(k.square(activation[:, 2: -2, 2: -2, :])) / scaling
    loss = loss + coeff * k.sum(k.square(activation)) / scaling


## Setting up the Gradient-ascent process

In [66]:
# This tensor holds the generated image: the dream
dream = model.input

# Computes the gradients of the dream with regard to the loss
grads = k.gradients(loss, dream)[0]

# Normalizes the gradients (important trick)
grads /= k.maximum(k.mean(k.abs(grads)), 1e-7)

# Sets up a Keras function to retrieve the value of the loss and gradients, given an input image
outputs = [loss, grads]
fetch_loss_and_grads = k.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# This function runs gradient ascent for a number of iterations.
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

 # The actual DeepDream algorithm

<img src="imgs/deepdream.png">
Finally: the actual DeepDream algorithm. 
    First, you define a list of scales (also called octaves) at which to process the images. 
    Each successive scale is larger than the previous one by a factor of 1.4 (it’s 40% larger): 
        you start by processing a small image and then increasingly scale it up (see figure 8.4).
        


### The DeepDream process: successive scales of spatial processing (octaves) and detail reinjection upon upscaling

For each successive scale, from the smallest to the largest, you run gradient ascent to maximize the loss you previously defined, at that scale. After each gradient ascent run, you upscale the resulting image by 40%.

To avoid losing a lot of image detail after each successive scale-up (resulting in increasingly blurry or pixelated images), you can use a simple trick: after each scale-up, you’ll reinject the lost details back into the image, which is possible because you know what the original image should look like at the larger scale.
    Given a small image size $S$ and a larger image size $L$ , you can compute the difference between the original image resized to size $L$ and the original resized to size $S$ —this difference quantifies the details lost when going from $S$ to $L$ .

### Running gradient ascent over different successive scales

In [62]:
# Import auxiliary functions
import sys, os
sys.path.insert(0, os.path.abspath('../utils'))
from utils import aux
import numpy as np

In [67]:


# Playing with these hyperparameters will let you achieve new effects.
step = 0.01          # Gradient ascent step size 
num_octave = 3       # Number of scales at which to run gradient ascent
octave_scale = 1.4   # Size ratio between scales

# Number of ascent steps to run at each scale
iterations = 20

# If the loss grows larger than 10, you’ll interrupt the gradient-ascent process to avoid ugly artifacts.
max_loss = 10.

# Fill this with the path to the image you want to use.
base_image_path = 'imgs/img_2.png' # cat.jpeg' 

# Loads the base image into a Numpy array (function is defined in aux.py)
img = aux.preprocess_image(base_image_path)

original_shape = img.shape[1:3]

# Prepares a list of shape tuples defining the different scales at which to run gradient ascent
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverses the list of shapes so they’re in increasing order
successive_shapes = successive_shapes[::-1]

# Resizes the Numpy array of the image to the smallest scale
original_img = np.copy(img)
shrunk_original_img = aux.resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    
    # Scales up the dream image
    img = aux.resize_img(img, shape)
    
    # Runs gradient ascent, altering the dream
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)    
    
    # Scales up the smaller version of the original image: it will be pixellated.
    upscaled_shrunk_original_img = aux.resize_img(shrunk_original_img, shape)

    # Computes the high-quality version of the original image at this size
    same_size_original = aux.resize_img(original_img, shape)
    
    # The difference between the two is the detail that was lost when scaling up.
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    # Reinjects lost detail into the dream
    img += lost_detail
    shrunk_original_img = aux.resize_img(original_img, shape)
    aux.save_img(img, fname='imgs/single_layer_contrib_value-3/mixed10_dream_at_scale_' + str(shape) + '.png')
    
    

Processing image shape (204, 362)
...Loss value at 0 : 1.7014828
...Loss value at 1 : 3.2517748
...Loss value at 2 : 4.887304
...Loss value at 3 : 5.0650344
...Loss value at 4 : 5.097887
...Loss value at 5 : 6.370093
...Loss value at 6 : 7.1348143
...Loss value at 7 : 8.050293
...Loss value at 8 : 7.7061176
Processing image shape (285, 507)
...Loss value at 0 : 1.6559529
...Loss value at 1 : 2.5954976
...Loss value at 2 : 3.967165
...Loss value at 3 : 4.95463
...Loss value at 4 : 4.3228474
...Loss value at 5 : 5.30603
...Loss value at 6 : 4.9186707
...Loss value at 7 : 7.7323685
...Loss value at 8 : 8.74538
Processing image shape (400, 711)
...Loss value at 0 : 1.6275585
...Loss value at 1 : 2.2773285
...Loss value at 2 : 2.3800404
...Loss value at 3 : 3.7075217
...Loss value at 4 : 3.6420116
...Loss value at 5 : 4.7930207
...Loss value at 6 : 5.099324
...Loss value at 7 : 5.8039517
...Loss value at 8 : 6.1651497
...Loss value at 9 : 6.3588643
...Loss value at 10 : 7.996332
...Loss val